In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn import linear_model
import scipy.stats as stats
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.precision", 4)

### 1 Conceptual issues for LTCM
1. Describe LTCM’s investment strategy in less than 100 words.

<span style="color:#00008B">**Solution:** Many reasonable things to put here. Should identify their reliance on relative value and convergence trades. Should also mention their opportunistic trading as they spot dislocations due to institutional demands. (For example, liquidity needs.)
</span>

2. What are LTCM’s biggest advantages over its competitors?

<span style="color:#00008B">**Solution:**
Several advantages.</span>  
<span style="color:#00008B">    
• Efficient financing. LTCM got very favorable terms on all financing—sometimes even zero haircut! Typically had small, if any, outlay.</span>
<span style="color:#00008B">    
• Fund size. Have market power even in the large market of institutional wholesale.</span>
<span style="color:#00008B">    
• Liquidity. LTCM has in place many mechanisms to ensure liquidity.</span>
<span style="color:#00008B">    
• Long-term horizon. In financing and assessing trades, LTCM takes a relatively long-term view.</span>
<span style="color:#00008B">    
• Hedged. LTCM avoids taking too much default risk or explicit directional bets.</span>
<span style="color:#00008B">   
Then again, LTCM went bust in Aug. 1998, so maybe these advantages were not as strong as it
seemed!
</span>  
    
3. The case discusses four types of funding risk facing LTCM:   
• collateral haircuts  
• repo maturity  
• equity redemption  
• loan access  
The case discusses specific ways in which LTCM manages each of these risks. Briefly discuss
them.

<span style="color:#00008B">**Solution:**   
The case discusses steps LTCM took to manage four types of funding risks.</span>  

<span style="color:#00008B">
1. Collateral haircuts. For most trades, LTCM obtains 100% financing on a fully collateralized
basis. Furthermore, LTCM stress tests the haircuts across its asset classes.</span>

<span style="color:#00008B">
2. Repo. LTCM goes against the norm by entering into relatively long-maturity repo. While much of it is overnight, LTCM uses contracts that typically have maturity of 6-12 months. Furthermore, LTCM manages their aggregate repo maturity.</span>

<span style="color:#00008B">
3. Equity redemption. The firm is highly levered, so equity funding risk is especially important. LTCM restricts redemptions of equity year by year. The restriction is particularly strong in that unredeemed money is re-locked.
They also spread the redemption windows across the year to ensure there is never a possi- bility of immediate withdrawal of a large portion of equity.</span>

<span style="color:#00008B">
4. For debt funding, LTCM negotiated a revolving loan that has no Material Adverse Change clause. Thus, the availability of debt funding is not so highly correlated with fund performance.  
</span>
    
4. LTCM is largely in the business of selling liquidity and volatility. Describe how LTCM accounts for liquidity risk in their quantitative measurements.

<span style="color:#00008B">**Solution:**  LTCM attempts to account for liquidity risk quantitatively by adjusting security correlations. For short-term horizons, LTCM assumes positive correlation between all trade cat- egories. Even if their net exposure to a strategy flips sides, they still assume positive correlation to the new net position.
Given the efforts of LTCM to hedge out obvious market risks, there are many strategies which would seem to have zero correlation. However, LTCM feels that liquidity concerns can cause the effective trading to be positively correlated.
</span>

5. Is leverage risk currently a concern for LTCM?

<span style="color:#00008B">**Solution:** It would seem that leverage is not particularly dangerous at the moment. The fund’s volatility is relatively low, its VaR is relatively low, nor is it particularly high relative to the rest of the industry.
Moreover, the firm actively manages its funding risk which theoretically means it should be able to handle the natural risks of high leverage.
At the time of the case, the firm is trying to determine whether to further increase leverage. Subsequently, at the end of 1997 the fund returned about a third of its 7.5 billion equity capital to investors.
Of course, less than a year later, the fund blew up, but from the time of the case it’s hard to see the leverage risk.
</span>

6. Many strategies of LTCM rely on converging spreads. LTCM feels that these are almost win/win situations because of the fact that if the spread converges, they make money. If it diverges, the trade becomes even more attractive, as convergence is still expected at a future date. What is the risk in these convergence trades?

<span style="color:#00008B">**Solution:** About a year after the time of the case, the fund loses most of its value due to non-converging trades. So clearly there is some risk!
Positions are subject to liquidity risk. If market liquidity dries up or the markets become segmented, the divergent spreads can persist for a long time. This indeed happens later to LTCM. The trades that get them in trouble ultimately pay off, but not before LTCM blows up.  
LTCM believes it can exit these convergence trades if they become too unprofitable. However, a stop-loss order is not the same as a put option. If the price jumps discontinuously through the stop-loss, then it is ineffective. Or a market may be paralyzed/illiquid when trying to execute the stop-loss. A put option does not need to worry about price impact, whereas a stop-loss does. Finally, a stop-loss ensures that an investor sells as soon as a security price hits a worst-case scenario, ensuring unfavorable market timing.
</span>

### 2 LTCM Risk Decomposition

In [2]:
ltcm = pd.read_excel('ltcm_exhibits_data.xlsx', sheet_name = "Sheet1").set_index('Date')
ltcm.head()

,Gross return,Net return
Date,,
1994-03-31,-0.011,-0.013
1994-04-30,0.014,0.008
1994-05-31,0.068,0.053
1994-06-30,-0.039,-0.029
1994-07-31,0.116,0.084


In [4]:
rf = pd.read_excel('gmo_analysis_data.xlsx', sheet_name = 3).set_index('Date')['1994-03-31':'1998-07-31']
rf.head()

,US3M
Date,
1994-03-31,0.0030
1994-04-30,0.0033
1994-05-31,0.0036
1994-06-30,0.0036
1994-07-31,0.0037


In [5]:
spy = pd.read_excel('gmo_analysis_data.xlsx', sheet_name = 2).set_index('Date')['1994-03-31':'1998-07-31']
ltcm = ltcm.join(spy['SPY'])
ltcm.head()

,Gross return,Net return,SPY
Date,,,
1994-03-31,-0.011,-0.013,-0.0419
1994-04-30,0.014,0.008,0.0112
1994-05-31,0.068,0.053,0.0159
1994-06-30,-0.039,-0.029,-0.0229
1994-07-31,0.116,0.084,0.0323


In [6]:
ex_ltcm = ltcm.subtract(rf['US3M'],axis = 'rows')
ex_ltcm.head()

,Gross return,Net return,SPY
Date,,,
1994-03-31,-0.0140,-0.0160,-0.0449
1994-04-30,0.0107,0.0047,0.0079
1994-05-31,0.0644,0.0494,0.0123
1994-06-30,-0.0425,-0.0326,-0.0264
1994-07-31,0.1123,0.0803,0.0287


In [7]:
def summary_stats(df, annual_fac):
    ss_df = (df.mean() * annual_fac).to_frame('Mean')
    ss_df['Vol'] = df.std() * np.sqrt(annual_fac)
    ss_df['Sharpe'] = ss_df['Mean'] / ss_df['Vol']
    #ss_df['VaR'] = df.quantile(0.05)
    return ss_df.T

In [8]:
def tail_risk_report(data, q):
    df = data.copy()
    df.index = data.index.date
    report = pd.DataFrame(columns = df.columns)
    
    report.loc['Skewness'] = df.skew()
    report.loc['Excess Kurtosis'] = df.kurtosis()
    report.loc['VaR'] = df.quantile(q)
    return report

1. Summary stats. 

(a) For both the gross and net series of LTCM excess returns, report the mean, volatility, and
Sharpe ratios. (Annualize them.)

(b) Report the skewness, kurtosis, and (historic) VaR(.05).

(c) Comment on how these stats compare to SPY and other assets we have seen. How much
do they differ between gross and net?

<span style="color:#00008B">**Solution:**
The return performances of both gross and net are very similar to those of SPY, but gross and net returns have more tail risk than SPY. Compared with other assets such as the FF-factors or hedge fund series we've seen so far, the Sharpe ratios of LTCM are much higher.   
The gross return has higher mean, slightly higher volatility, and higher Sharpe ratio than the net return. The gross return also has less negative skewness, less excess kurtosis, and smaller VaR than the net return. All these stats show that gross return looks more attractive than the net return. 
</span>

In [9]:
# 1
summary_stats(ex_ltcm,12)

,Gross return,Net return,SPY
Mean,0.2421,0.1554,0.1738
Vol,0.1362,0.1118,0.1123
Sharpe,1.7769,1.3901,1.5479


In [10]:
tail_risk_report(ex_ltcm,0.05)

,Gross return,Net return,SPY
Skewness,-0.2877,-0.8102,-0.4335
Excess Kurtosis,1.5866,2.9269,-0.3620
VaR,-0.0304,-0.0264,-0.0464


2. Using the series of net LTCM excess returns, denoted $\tilde{r}^{LTCM}$, estimate the following regression:

(a) Report $\alpha$ and $\beta_m$. Report the $R_2$ stat.

(b) From this regression, does LTCM appear to be a “closet indexer”?

<span style="color:#00008B">**Solution:**
LTCM is definitely not a closet indexer - at least with respect to the market index. The market factor explains only 0.019 of the fund’s return variation. Furthermore,  the response to the market is quite small on average as the market beta is only 0.1371. 
</span>

(c) From the regression, does LTCM appear to deliver excess returns beyond the risk premium
we expect from market exposure?

<span style="color:#00008B">**Solution:**
Yes. LTCM has market exposure of $\beta_m=0.1371$. If this were the only channel of delivering risk premium, then LTCM would have a tiny risk premium. However, the summary stats show it has a relatively large mean excess return - much larger than 0.1371 multiplied by the market risk premium.  
More simply, one could note that LTCM and the market are both traded securities, yet LTCM delivers mean excess return beyond the market given that it has positive $\alpha$ of 0.011 per month.
</span>

In [11]:
lhs = ex_ltcm['Net return']
rhs = sm.add_constant(ex_ltcm['SPY'])
res = sm.OLS(lhs, rhs, missing='drop').fit()
single_report = res.params.to_frame('Single')
single_report.loc['R_squared','Single'] = res.rsquared
single_report

,Single
const,0.0110
SPY,0.1371
R_squared,0.0190


3. Let’s check for non-linear market exposure. Run the following regression on LTCM’s net excess returns:

(a) Report $\beta_1$, $\beta_2$, and the $R_2$ stat.

(b) Does the quadratic market factor do much to increase the overall LTCM variation explained by the market?

<span style="color:#00008B">**Solution:**
Yes. The $R_2$ goes from .0190 to .0243. Still, the quadratic market model leaves much of LTCM unexplained.
</span>

(c) From the regression evidence, does LTCM’s market exposure behave as if it is long market options or short market options?

<span style="color:#00008B">**Solution:**
The negative quadratic beta tells us that LTCM’s return is, on average, concave in the market return. That is, LTCM is less than proportionally up with the market and more than proportionally down when the market is down. This is the type of exposure LTCM would get by selling put options and call options on the market.
</span>

(d) Should we describe LTCM as being positively or negatively exposed to market volatility?

<span style="color:#00008B">**Solution:**
LTCM is hurt by market volatility given that the quadratic term implies low LTCM returns when the market is particularly high or low.
</span>

In [12]:
# 3
X = pd.DataFrame(columns = ['SPY','SPY Squared'],index = ex_ltcm.index)
X['SPY'] =  ex_ltcm['SPY']
X['SPY Squared'] = ex_ltcm['SPY']**2
lhs = ex_ltcm['Net return']
rhs = sm.add_constant(X)
res = sm.OLS(lhs, rhs, missing='drop').fit()
quad_report = res.params.to_frame('Quadratic')
quad_report['P-values'] = res.pvalues
quad_report.loc['R_squared','Quadratic'] = res.rsquared
quad_report

,Quadratic,P-values
const,0.0129,0.0412
SPY,0.1669,0.2719
SPY Squared,-1.9267,0.6031
R_squared,0.0243,NaN


4. Let’s try to pinpoint the nature of LTCM’s nonlinear exposure. Does it come more from exposure to up-markets or down-markets? Run the following regression on LTCM’s net excess returns:

(a) Report $\beta$, $\beta_u$, $\beta_d$, and the $R_2$ stat.

(b) Is LTCM long or short the call-like factor? And the put-like factor?

<span style="color:#00008B">**Solution:**
LTCM is short a call option as seen by the negative estimate of $\beta_u$. LTCM also has a positive $\beta_d$ which would appear to give it a long put exposure.
</span>

(c) Which factor moves LTCM more, the call-like factor, or the put-like factor?

<span style="color:#00008B">**Solution:**
The put-like factor influences LTCM more as the magnitude of $\beta_u$ is less than 1 and the magnitude of $\beta_d$ is greater than 1. 
</span>

(d) In the previous problem, you commented on whether LTCM is positively or negatively exposed to market volatility. Using this current regression, does this volatility exposure come more from being long the market’s upside? Short the market’s downside? Something else?

<span style="color:#00008B">**Solution:**
This regression shows that LTCM is neither levered to the downside nor exposed upside of the market, which agrees with the previous comment. 
</span>

In [13]:
# 4
k1 = 0.03
k2 = -0.03

In [14]:
X = pd.DataFrame(columns = ['call','put'],index = ex_ltcm.index)
X['call'] = (ex_ltcm['SPY'] - k1).clip(lower = 0)
X['put'] = (k2 - ex_ltcm['SPY']).clip(lower = 0)
X['SPY'] = ex_ltcm['SPY']
lhs = ex_ltcm['Net return']
rhs = sm.add_constant(X)
res = sm.OLS(lhs, rhs, missing='drop').fit()
option_report = res.params.to_frame('Option')
option_report['P-values'] = res.pvalues
option_report.loc['R_squared','Option'] = res.rsquared
option_report

,Option,P-values
const,0.0084,0.1629
call,-0.7821,0.2193
put,1.2896,0.2705
SPY,0.4666,0.0980
R_squared,0.0555,NaN


### 3 The FX Carry Trade

In [15]:
rf_rates = pd.read_excel('fx_carry_data.xlsx', sheet_name = 1)
rf_rates = rf_rates.set_index('DATE')
log_rf_rates = np.log(1+rf_rates)
log_rf_rates

,USD3M,GBP3M,EUR3M,CHF3M,JPY3M
DATE,,,,,
1999-01-31,0.0485,0.0565,0.0303,0.0126,0.0048
1999-02-28,0.0490,0.0532,0.0305,0.0125,0.0027
1999-03-31,0.0488,0.0519,0.0292,0.0124,0.0019
1999-04-30,0.0487,0.0519,0.0255,0.0099,0.0013
1999-05-31,0.0494,0.0523,0.0255,0.0102,0.0009
...,...,...,...,...,...
2021-06-30,0.0015,0.0008,-0.0055,-0.0076,-0.0008
2021-07-31,0.0012,0.0007,-0.0056,-0.0076,-0.0010
2021-08-31,0.0012,0.0007,-0.0056,-0.0077,-0.0010


In [16]:
fx_rates = pd.read_excel('fx_carry_data.xlsx', sheet_name = 2)
fx_rates = fx_rates.set_index('DATE')
log_fx_rates = np.log(fx_rates)
log_fx_rates

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.4982,0.1285,-0.3484,-4.7536
1999-02-28,0.4717,0.0949,-0.3712,-4.7766
1999-03-31,0.4787,0.0777,-0.3904,-4.7743
1999-04-30,0.4753,0.0549,-0.4225,-4.7827
1999-05-31,0.4713,0.0413,-0.4240,-4.7948
...,...,...,...,...
2021-06-30,0.3225,0.1696,0.0776,-4.7100
2021-07-31,0.3302,0.1709,0.0986,-4.6977
2021-08-31,0.3182,0.1655,0.0876,-4.7009


In [17]:
sub_rf = log_rf_rates.iloc[:,1:]
sub_rf.columns = log_fx_rates.columns
sub_rf

,USUK,USEU,USSZ,USJP
DATE,,,,
1999-01-31,0.0565,0.0303,0.0126,0.0048
1999-02-28,0.0532,0.0305,0.0125,0.0027
1999-03-31,0.0519,0.0292,0.0124,0.0019
1999-04-30,0.0519,0.0255,0.0099,0.0013
1999-05-31,0.0523,0.0255,0.0102,0.0009
...,...,...,...,...
2021-06-30,0.0008,-0.0055,-0.0076,-0.0008
2021-07-31,0.0007,-0.0056,-0.0076,-0.0010
2021-08-31,0.0007,-0.0056,-0.0077,-0.0010


1. The Static Carry Trade

Report the following stats, (based on the excess log returns.) Annualize them.  
(a) mean  
(b) volatility  
(c) Sharpe ratio

In [18]:
# 1
ex_carry = (log_fx_rates - log_fx_rates.shift(1) + sub_rf).subtract(log_rf_rates['USD3M'],axis = 'rows')

In [19]:
summary_stats(ex_carry,12)

,USUK,USEU,USSZ,USJP
Mean,0.0474,-0.0603,-0.1590,-0.2204
Vol,0.0963,0.1079,0.1107,0.1111
Sharpe,0.4925,-0.5584,-1.4361,-1.9846


2. Implications for UIP:

(a) Do any of these stats contradict the (log version) of Uncovered Interest Parity (UIP)?

<span style="color:#00008B">**Solution:**
According to UIP, currency appreciation will counterbalance rate differentials, on average. Thus, the mean excess returns would all be zero, as well as the Sharpe ratios.
However, currency appreciation would not necessarily counterbalance rate differentials for any given month—just on average. Thus, non-zero volatility, skewness, and kurtosis do not contradict UIP.
</span>

(b) A long position in which foreign currency offered the best Sharpe ratio over the sample?

<span style="color:#00008B">**Solution:**
The Sharpe ratio is largest for holding GBP.
</span>

(c) Are there any foreign currencies for which a long position earned a negative excess return (in USD) over the sample?

<span style="color:#00008B">**Solution:**
Over the sample, holding EUR, CHF, and JPY will give a negative excess return. 
</span>


3. Predicting FX

(a) Make a table with 6 columns—each corresponding to a different currency regression. Report the regression estimates $\alpha_i$ and $\beta_i$ in the first two rows. Report the $R_2$ stat in the third row.

(b) Suppose the foreign risk-free rate increases relative to the US rate.

i. For which foreign currencies would we predict a relative strengthening of the USD in the following period?

<span style="color:#00008B">**Solution:**
JPY has positive regression slopes. Thus, when the currency has relatively large riskless rates relative to the USD, we predict relatively more USD appreciation.
</span>

ii. For which currencies would we predict relative weakening of the USD in the following period?

<span style="color:#00008B">**Solution:**
GBP, EUR, and CHF have negative regression slopes. Thus, when the currency has relatively large riskless rates relative to the USD, we predict relatively more USD depreciation.
</span>

iii. This FX predictability is strongest in the case of which foreign currency?

<span style="color:#00008B">**Solution:**
According to the $R_2$ stats, the predictability of FX rates is much stronger for CHF and EUR relative to the other four currencies. Even so, FX predictability is relatively small—there is a lot of variation in FX rates unexplained by interest rate differentials.
</span>

In [20]:
# 3
Y = log_fx_rates - log_fx_rates.shift(1)
X = - sub_rf.subtract(log_rf_rates['USD3M'],axis = 'rows')

In [21]:
ols_report = pd.DataFrame(index=log_fx_rates.columns)

for col in log_fx_rates.columns:
    lhs = Y[col]
    rhs = sm.add_constant(X[col])
    res = sm.OLS(lhs, rhs, missing='drop').fit()
    ols_report.loc[col, 'alpha'] = res.params['const'] * 12
    ols_report.loc[col, 'beta'] = res.params[col]
    ols_report.loc[col, 'R_squared'] = res.rsquared
ols_report.T

,USUK,USEU,USSZ,USJP
alpha,-8.8634e-03,0.0091,0.0517,-1.5757e-03
beta,-1.3670e-02,-0.1381,-0.1828,1.0527e-02
R_squared,4.4435e-05,0.0046,0.0065,5.3817e-05


4. The Dynamic Carry Trade

(a) Use your regression estimates from Problem 3 along with the formula above to calculate the fraction of months for which the estimated FX risk premium positive. 

(b) Which currencies most consistently have a positive FX risk premium? And for which
currencies does the FX risk premium most often go negative?

<span style="color:#00008B">**Solution:**
The table indicates the fraction of months for which the dynamic carry trade would have recommended a long position in the foreign currency (against the USD.) Note that for the CHF, the interest rate differentials were favorable enough that the dynamic strategy would have gone long this currency in every month.  
For JPY, on the other hand, the strategy would not have long at all.
</span>

(c) Explain how we could use these conditional risk premia to improve the static carry trade returns calculated in Problem 1.

<span style="color:#00008B">**Solution:**
We can use these conditional risk premia to help us decide whether and when to enter the carry trade. 
</span>

In [22]:
# 4
estimate_fx = (ols_report['beta'] - 1) * X + ols_report['alpha']

In [23]:
freq_report = pd.DataFrame(index=log_fx_rates.columns)
for col in log_fx_rates.columns:
    freq_report.loc[col,'Frequency of positive weight'] = (estimate_fx[col] > 0).sum() / len(estimate_fx)
freq_report.T

,USUK,USEU,USSZ,USJP
Frequency of positive weight,0.2555,0.6168,1.0,0.0
